In [23]:


import requests
import os
import string

# variables 
CWD = os.getcwd()
LOCATION = os.path.join(CWD,'MUSIC')
if os.path.isdir(LOCATION)==False:
    os.mkdir(LOCATION)


Medium article : "

#### Functions

In [24]:
def get_ID(session, id):
    LINK = f'https://api.spotifydown.com/getId/{id}'
    headers = {
        'authority': 'api.spotifydown.com',
        'method': 'GET',
        'path': f'/getId/{id}',
        'origin': 'https://spotifydown.com',
        'referer': 'https://spotifydown.com/',
        'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        'sec-fetch-mode': 'cors',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }
    response = session.get(url = LINK, headers=headers)
    if response.status_code == 200 : 
        data = response.json()
        return data
    return None

def generate_Analyze_id(session, yt_id):

    DL      = 'https://corsproxy.io/?https://www.y2mate.com/mates/analyzeV2/ajax'
    data    = {
        'k_query': f'https://www.youtube.com/watch?v={yt_id}',
        'k_page': 'home',
        'hl': 'en',
        'q_auto': 0,
    }
    headers = {
            'authority': 'corsproxy.io',
            'method': 'POST',
            'path': '/?https://www.y2mate.com/mates/analyzeV2/ajax',
            'origin': 'https://spotifydown.com',
            'referer': 'https://spotifydown.com/',
            'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
            'sec-fetch-mode': 'cors',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
    RES = session.post(url=DL, data=data, headers=headers)
    if RES.status_code ==200:
        return RES.json()
    return None

def generate_Conversion_id(session,  analyze_yt_id, analyze_id):

    DL      = 'https://corsproxy.io/?https://www.y2mate.com/mates/convertV2/index'
    data    = {
        'vid'   : analyze_yt_id,
        'k'     : analyze_id,
    }
    headers = {
            'authority': 'corsproxy.io',
            'method': 'POST',
            'path': '/?https://www.y2mate.com/mates/analyzeV2/ajax',
            'origin': 'https://spotifydown.com',
            'referer': 'https://spotifydown.com/',
            'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
            'sec-fetch-mode': 'cors',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
  
    RES = session.post(url=DL, data=data, headers=headers)
    if RES.status_code ==200:
        return RES.json()
    return None
def returnSPOT_ID(link):
    return link.split('/')[-1].split('?si')[0]

#### Start of Program

In [25]:
SPOTIFY_PLAYLIST_LINK = input('Spotify Link : ')


ID = returnSPOT_ID(SPOTIFY_PLAYLIST_LINK)
print('[*] SPOTIFY PLAYLIST ID    : ',ID)

Playlist_Link = f'https://api.spotifydown.com/trackList/playlist/{ID}'

session = requests.Session()

offset_data = {}


response = session.get(url = Playlist_Link)
offset = 0
while offset != None :
    if response.status_code == 200 : 
        Tdata = response.json()['trackList']
        page = response.json()['nextOffset']
        for count,song in enumerate(Tdata):
            yt_id = get_ID(session=session, id=song['id'])
            
            filename = song['title'].translate(str.maketrans('', '', string.punctuation)) + ' - ' + song['artists'].translate(str.maketrans('', '', string.punctuation)) + '.mp3'
            print('*'*25, str(count+1) + '/' + str(len(Tdata)), '*'*25)
            print('[*] Name of Song         : ', song['title'])
            print('[*] Spotify ID of Song   : ',song['id'])
            print('[*] Youtube ID of Song   : ',yt_id['id'])
            data  = generate_Analyze_id(session = session, yt_id = yt_id['id'])
            DL_ID = data['links']['mp3']['mp3128']['k']
            DL_DATA = generate_Conversion_id(session= session,  analyze_yt_id = data['vid'], analyze_id = DL_ID )
            DL_LINK = DL_DATA['dlink']
            ## DOWNLOAD
            link= session.get(DL_LINK)
            ## Save
            with open(os.path.join(LOCATION, filename), 'wb') as f:
                f.write(link.content)
    if page!=None:
        offset_data['offset'] = page
        response = session.get(url = Playlist_Link, params=offset_data)
    else:
        break

[*] SPOTIFY PLAYLIST ID    :  37i9dQZF1DWWDLvD3P5wOG
************************* 1/80 *************************
[*] Name of Song         :  Gasolina
[*] Spotify ID of Song   :  228BxWXUYQPJrJYHDLOHkj
[*] Youtube ID of Song   :  46azAz0AWv8
************************* 2/80 *************************
[*] Name of Song         :  Pon de Replay
[*] Spotify ID of Song   :  4TsmezEQVSZNNPv5RJ65Ov
[*] Youtube ID of Song   :  9fT1CEf7Wzg
************************* 3/80 *************************
[*] Name of Song         :  Temperature
[*] Spotify ID of Song   :  0k2GOhqsrxDTAbFFSdNJjT
[*] Youtube ID of Song   :  W0g_9OGdH-o
************************* 4/80 *************************
[*] Name of Song         :  It Wasn't Me
[*] Spotify ID of Song   :  3WkibOpDF7cQ5xntM1epyf
[*] Youtube ID of Song   :  sTMgX1PDGAE
************************* 5/80 *************************
[*] Name of Song         :  Sweat (A La La La La Long)
[*] Spotify ID of Song   :  1SssFw74DdHVjRa6ADggdD
[*] Youtube ID of Song   :  5M65

ConnectionError: HTTPSConnectionPool(host='dl165.dlmate03.online', port=443): Max retries exceeded with url: /?file=M3R4SUNiN3JsOHJ6WWQ2a3NQS1Y5ZGlxVlZIOCtyZ0dpc0F4akQ4bEIrQjFzNFZrOWVXaktJZENQdlJKeTRPdUZPRkU5Q3JRTEpYWkZoYUdvNHd6UUhiRXBmQXdxV3lLb2Fod0hKdFhUMFQxbXZLbTJESjVoRXk0TmRyZkFMTnBZbkY5bkZsMjJ4bVJ6ZkxmaGlyTGdXKzA5bG1XVHpVSi9EWWJiclBYOUpBVjRWMk5hdS8wMm9JQXVTK1M1OGNZaktmSTdWQ2hrdU5yL3M5OFJFRXBJY2dOZ00ydGhyYmZzeG9kZ3NvPQ%3D%3D (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018D456B9720>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))